In [1]:
import pandas as pd
import numpy as np

data = pd.read_parquet('productos_con_embeddings.parquet')

# data.sample(n=3)

In [2]:
from chromadb import PersistentClient
client = PersistentClient(path="./chroma_db")
collection = client.get_or_create_collection(name="productos")




In [3]:
i = 0
# Insertar registros en ChromaDB
for idx, row in data.iterrows():
    # Asegurarse de que el embedding esté en formato de lista
    # embedding = json.loads(row["embedding"]) if isinstance(row["embedding"], str) else row["embedding"]
    i += 1
    collection.add(
        ids=[str(idx)],  # ID único para cada registro
        documents=[f"{row['product_name']} {row['descripcion_producto']} {' '.join(row['categories'])}"],
        metadatas=[{
            "producto": row["product_name"],
            "enlace": row["link"],
            "precio": row["price"],
            "mercado": row["source"],
            "descripcion": row["descripcion_producto"]
        }],
        embeddings=[np.array([0.1, 0.2, 0.3])]  # Embedding pre-generado
    )
    if i > 5:
        break


In [4]:
# Recuperar un registro por su ID
result = collection.get(ids=["0"])

# Imprimir el registro
print(result)


{'ids': ['0'], 'embeddings': None, 'documents': ['Agua Tónica BRITVIC Paquete 12un Lata 150ml .Agua tónica Marca: BRITVIC  Contenido: 12un de 150ml  .AGUASplazaVea presenta una gran variedad de Bebidas para que elijas tu favorita de acuerdo a diferentes situaciones u ocasiones especiales. Hidratarse es muy importante para el correcto cuidado de nuestro organismo, además que nos ayuda a lucir mejor, porque una piel hidratada se refleja siempre en el exterior. Por tal motivo, te ofrecemos diferentes presentaciones de Aguas: agua en sixpack, agua en bidón, agua personal, agua de 2.5 litros y más.Puedes elegir entre Agua Mineral y Agua de Mesa en las marcas más populares del mercado. También puedes revisar nuestras opciones de Aguas Saborizadas para que te hidrates con una opción más rica pero libre de azúcares.  Bebidas Aguas Agua Tónica'], 'uris': None, 'data': None, 'metadatas': [{'descripcion': '.Agua tónica Marca: BRITVIC  Contenido: 12un de 150ml  .AGUASplazaVea presenta una gran var

In [ ]:
def visualize_chromadb_data(collection, limit=10):
    """
    Visualiza la información almacenada en una colección de ChromaDB.
    
    Args:
        collection: La colección de ChromaDB que deseas visualizar.
        limit: Número máximo de registros a mostrar (por defecto 10).
        
    Returns:
        None. Imprime los datos en formato tabular.
    """
    # Recupera todos los IDs de la colección
    ids = collection.get()['ids'][:limit]
    
    # Inicializa listas para guardar los datos
    data_rows = []
    
    for i in ids:
        # Recupera los datos de cada ID
        record = collection.get(ids=[i])
        document = record.get('documents', [None])[0] if record.get('documents') else "Sin documento"
        metadata = record.get('metadatas', [None])[0] if record.get('metadatas') else "Sin metadatos"
        embedding = record.get('embeddings', [None])[0] if record.get('embeddings') else "Sin embedding"
        
        # Agrega una fila a la lista
        data_rows.append({
            'ID': i,
            'Documento': document,
            'Metadatos': metadata,
            'Embedding': embedding
        })
    
    # Convierte los datos a un DataFrame para mejor visualización
    df = pd.DataFrame(data_rows)
    print(df.to_string(index=False))  # Imprime el DataFrame sin índice

# Ejemplo de uso
visualize_chromadb_data(collection)

In [ ]:
import json
i=0
# Insertar registros en ChromaDB
for idx, row in data.iterrows():
    i += 1
    print(type(row["embedding"]))
    print(row["embedding"][0:5].tolist())

    # Asegurarse de que el embedding esté en formato de lista
    embedding = json.loads(row["embedding"]) if isinstance(row["embedding"], str) else row["embedding"]
    # print(embedding)
    if i > 5:
        break
    
    
    # collection.add(
    #     ids=[str(idx)],  # ID único para cada registro
    #     documents=[f"{row['product_name']} {row['descripcion_producto']} {' '.join(row['categories'])}"],
    #     metadatas=[{
    #         "producto": row["product_name"],
    #         "enlace": row["link"],
    #         "precio": row["price"],
    #         "mercado": row["source"],
    #         "descripcion": row["descripcion_producto"]
    #     }],
    #     embeddings=[embedding]  # Embedding pre-generado
    # )


In [ ]:
def visualize_chromadb_with_valid_embeddings(collection, limit=10):
    """
    Visualiza los registros de ChromaDB que tienen embeddings válidos.
    
    Args:
        collection: La colección de ChromaDB que deseas visualizar.
        limit: Número máximo de registros a mostrar (por defecto 10).
        
    Returns:
        None. Imprime los datos en formato tabular.
    """
    # Recupera todos los IDs de la colección
    ids = collection.get()['ids']
    
    # Inicializa listas para guardar los datos
    data_rows = []
    count = 0
    
    for i in ids:
        # Recupera los datos de cada ID
        record = collection.get(ids=[i])
        embedding = record.get('embeddings', [None])[0] if record.get('embeddings') else None
        
        # Solo procesar registros con embeddings válidos
        if embedding is not None:
            document = record.get('documents', [None])[0]
            metadata = record.get('metadatas', [None])[0]
            
            # Agrega una fila a la lista
            data_rows.append({
                'ID': i,
                'Documento': document if document else "Sin documento",
                'Metadatos': metadata if metadata else "Sin metadatos",
                'Embedding': embedding
            })
            
            count += 1
            if count >= limit:
                break
    
    # Convierte los datos a un DataFrame para mejor visualización
    if data_rows:
        df = pd.DataFrame(data_rows)
        print(df.to_string(index=False))  # Imprime el DataFrame sin índice
    else:
        print("No hay registros con embeddings válidos.")

# Ejemplo de uso
visualize_chromadb_with_valid_embeddings(collection, limit=10)
